# How to customize or contribute to the knowledge base

The [asp](https://github.com/cmudig/draco2/tree/main/draco/asp) directory contains the Draco knowledge base as answer set programs. 

## draco asp files
 * `define.lp` declares the domains to visualization attributes.
 * `helper.lp` defines useful helper functions. 
 * `generate.lp` sets up the search space.
 * `constraints.lp` restricts the search space to follow the correct draco general description language. 
 * `hard.lp` restricts the search space to only well-formed and expressive specifications.
 * `soft.lp` defines soft constraints in the form of violation/1 and violation/2 predicates. By themselves, these predicates don't change the search.


## guideline
You can design your own description language and use it with Draco or extend the existing language we use here. If you don't know where to start with the constraints, you can first set up the search space in `generate.lp` and use our `run_clingo` API to generate some recommendations. Then, you should be able to find some recommendations that should have been left out, and you can write constraints to reflect them.  

For example the following snippet shows how to use the `run_clingo` API to generate 1 recommendations.

In [1]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo
from draco.programs import define, hard, helpers, constraints, generate
from pprint import pprint

prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
)
with open("../../draco/asp/examples/scatter.lp") as file:
    scatter = file.read()
for model in run_clingo(prog + scatter + ":- {entity(mark,_,_)}<2.", 100):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    print("\n")

{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'value',
 'view': [{'mark': [{'encoding': [{'aggregate': 'stdev',
                                   'channel': 'y',
                                   'field': 'f0'},
                                  {'aggregate': 'stdev',
                                   'channel': 'x',
                                   'field': 'f1'},
                                  {'aggregate': 'count', 'channel': 'text'}],
                     'type': 'text'},
                    {'encoding': [{'channel': 'text', 'field': 'f1'}],
                     'type': 'text'}],
           'scale': [{'channel': 'y', 'type': 'linear'},
                     {'channel': 'text', 'type': 'linear'},
                     {'channel': 'x', 'type': 'linear'}]}]}
[attribute(number_rows,root,100), attribute((field,name),f0,temperature), attribute((field,type),f0,number), attribute((field,nam

{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'value',
 'view': [{'mark': [{'encoding': [{'aggregate': 'sum',
                                   'channel': 'size',
                                   'field': 'f0'},
                                  {'binning': 25,
                                   'channel': 'detail',
                                   'field': 'f1'},
                                  {'aggregate': 'stdev',
                                   'channel': 'text',
                                   'field': 'f0'}],
                     'type': 'text'},
                    {'encoding': [{'binning': 10,
                                   'channel': 'detail',
                                   'field': 'f0'}],
                     'type': 'rect'}],
           'scale': [{'channel': 'text', 'type': 'ordinal'},
                     {'channel': 'size', 'type': 'ordinal', 'zero': 'true'

In [2]:
for model in run_clingo("a. b :- a."):
    print(model)

    print(model.answer_set[0])
    print(model.answer_set[1])

a.
b.
a
b


In [3]:
import draco
import importlib

importlib.reload(draco)

<module 'draco' from '/Users/junranyang/Documents/code/draco2/draco/__init__.py'>

In [4]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo
from draco.programs import define, hard, helpers, constraints, generate
from pprint import pprint

prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
)
with open("../../draco/asp/examples/scatter.lp") as file:
    scatter = file.readlines()
scatter = ("").join(scatter)
for model in run_clingo(prog + scatter, 10, True):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    print("\n")
# model = next(run_clingo(prog + scatter))
# pprint (answer_set_to_dict(model.answer_set))
# print (model.answer_set)

In [5]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo
from draco.programs import define, hard, helpers, constraints, generate
from pprint import pprint

prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
)
with open("../../draco/asp/examples/scatter.lp") as file:
    scatter = file.readlines()
scatter = ("").join(scatter)
for model in run_clingo(
    prog + scatter + ":- {entity(view,_,_)} < 3.", 10, False, {"max_marks": 3}
):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    print("\n")

TypeError: can only concatenate list (not "dict") to list

In [ ]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo
from draco.programs import define, hard, helpers, constraints, generate
from draco.asp_utils import blocks_to_program
from pprint import pprint

c = "".join(
    blocks_to_program(
        constraints.blocks, set(constraints.blocks.keys()) - set(["violation"])
    )
)
prog = generate.program + define.program + helpers.program + hard.program + c
with open("../../draco/asp/examples/scatter.lp") as file:
    scatter = file.readlines()
scatter = ("\n").join(scatter)
for model in run_clingo(prog + scatter, 100):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    print("\n")

In [ ]:
answer = [str(symbol) + ". " for symbol in model.answer_set]

print(answer)
print(type(answer))

In [ ]:
from draco import get_violations

print(get_violations(answer))

In [ ]:
prog = generate.program + scatter
print(get_violations(prog))

In [ ]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo, get_violations
from draco.process_weights import weights
from collections import defaultdict


from draco.programs import (
    define,
    hard,
    helpers,
    constraints,
    generate,
    soft,
    optimize,
)
from pprint import pprint


prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
    + optimize.program
    + weights.weights_program
    + weights.assign_program
    + soft.program
)
with open("../../draco/asp/examples/scatter.lp") as file:
    scatter = file.readlines()
scatter = ("").join(scatter)
print(get_violations(scatter))
for model in run_clingo(prog + scatter, 30, True):
    result = defaultdict(int)
    for symbol in model.answer_set:
        if symbol.name == "preference":
            result[symbol.arguments[0].name] += 1

    print(result)
    print(model.cost)

    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    print("\n")

In [ ]:
from typing import Iterable, Union, List, Optional, Dict


def count_preferences(draco_query: List[str], debug=False) -> Optional[Dict[str, int]]:
    """Get a dictionary of violations for a full spec.
    Args:
        task: a task spec object
    Returns:
        a dictionary storing violations of soft rules
    """
    program = (
        ("").join(draco_query)
        + define.program
        + helpers.program
        # + hard.program
        + constraints.program
        + weights.weights_program
        + weights.assign_program
        + soft.program
    )

    # print(get_violations(draco_query))
    # print(draco_query)
    # assert len(get_violations(draco_query)) == 0

    result = defaultdict(int)

    model = next(run_clingo(program))

    for symbol in model.answer_set:
        if symbol.name == "preference":
            result[symbol.arguments[0].name] += 1

    return result

In [ ]:
from draco import dict_to_facts, answer_set_to_dict, run_clingo
from draco.process_weights import weights
from collections import defaultdict


from draco.programs import (
    define,
    hard,
    helpers,
    constraints,
    generate,
    soft,
    optimize,
)
from pprint import pprint


prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
    + optimize.program
    + weights.weights_program
    + weights.assign_program
    + soft.program
)
with open("../../draco/asp/examples/scatter.lp") as file:
    scatter = file.readlines()
scatter = ("").join(scatter)
print(get_violations(scatter))
for model in run_clingo(prog + scatter, 500, True):
    result = defaultdict(int)
    for symbol in model.answer_set:
        if symbol.name == "preference":
            result[symbol.arguments[0].name] += 1

    print(result)
    print(model.cost)

    answer = [str(symbol) + ". " for symbol in model.answer_set]
    #     print(get_violations(answer))

    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    print("\n")

In [6]:
from draco import Draco, answer_set_to_dict
from pprint import pprint

d = Draco()

with open("../../draco/asp/examples/scatter.lp") as f:
    hist_spec = f.read()

print("INPUT:")
print(hist_spec)

print("OUTPUT:")
model = next(d.complete_spec(hist_spec))
pprint(answer_set_to_dict(model.answer_set))
pprint(str(model.answer_set))

print("VIOLATED PREFERENCES:")
pprint(d.count_preferences(str(model)))

INPUT:
attribute(number_rows,root,100).

entity(field,root,f0).
attribute((field,name),f0,temperature).
attribute((field,type),f0,number).

entity(field,root,f1).
attribute((field,name),f1,precipitation).
attribute((field,type),f1,number).

entity(view,root,v0).

entity(mark,v0,m).
entity(encoding,m,e0).
attribute((encoding,field),e0,f0).
entity(encoding,m,e1).
attribute((encoding,field),e1,f1).

#show entity/3.
#show attribute/3.

OUTPUT:
{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'mark': [{'encoding': [{'channel': 'y', 'field': 'f0'},
                                  {'channel': 'x', 'field': 'f1'}],
                     'type': 'point'}],
           'scale': [{'channel': 'y', 'type': 'linear', 'zero': 'true'},
                     {'channel': 'x', 'type': 'linear', 'zero': 'true'}]}]}
('[attribute(number_rows,root,100), attribute((field,name),f0,temperature), '
 '